In [1]:
from src.metagpt import BioformatsReader
import javabridge
import bioformats
import ome_types
from src.metagpt.utils import _init_logger

javabridge.start_vm(class_path=bioformats.JARS)
_init_logger()

image_path = "/home/aaron/Documents/Projects/MetaGPT/in/images/small_images/Rio9 0528.tif"
ome_xml = BioformatsReader.get_omexml_metadata(image_path)
ome_raw = BioformatsReader.get_raw_metadata(image_path)
ome_tree = BioformatsReader.raw_to_tree(ome_raw)
ome_dict = ome_types.to_dict(ome_xml)

javabridge.kill_vm()

{'XResolution': '2728469.0', 'PhotometricInterpretation': 'BlackIsZero', 'PlanarConfiguration': 'Chunky', 'MetaDataPhotometricInterpretation': 'Monochrome', 'ImageLength': '3072', 'BitsPerSample': '16', 'ResolutionUnit': 'Inch', 'Compression': 'LZW', 'SamplesPerPixel': '1', 'NumberOfChannels': '1', 'MetaMorph': 'no', 'ImageWidth': '3072', 'Orientation': '1st row -> top; 1st column -> left', 'YResolution': '2728469.0'}
{}
{'XResolution': '2728469.0', 'PhotometricInterpretation': 'BlackIsZero', 'PlanarConfiguration': 'Chunky', 'MetaDataPhotometricInterpretation': 'Monochrome', 'ImageLength': '3072', 'BitsPerSample': '16', 'ResolutionUnit': 'Inch', 'Compression': 'LZW', 'SamplesPerPixel': '1', 'NumberOfChannels': '1', 'MetaMorph': 'no', 'ImageWidth': '3072', 'Orientation': '1st row -> top; 1st column -> left', 'YResolution': '2728469.0'}


AttributeError: 'IFormatReader' object has no attribute '__annotations__'

In [3]:
for k,v in ome_tree.items():
    print(k, v)

XResolution 2728469.0
PhotometricInterpretation BlackIsZero
PlanarConfiguration Chunky
MetaDataPhotometricInterpretation Monochrome
ImageLength 3072
BitsPerSample 16
ResolutionUnit Inch
Compression LZW
SamplesPerPixel 1
NumberOfChannels 1
MetaMorph no
ImageWidth 3072
Orientation 1st row -> top; 1st column -> left
YResolution 2728469.0


In [2]:
# One shot prediction of the structured annotations node
from src.metagpt.predictors.predictor_xml_annotation import PredictorXMLAnnotation
from ome_types._autogenerated.ome_2016_06 import XMLAnnotation, OME
from src.metagpt.utils import dict_to_xml_annotation
import ome_types

predictor = PredictorXMLAnnotation(str(ome_tree))
prediction = predictor.predict()
print(prediction)

#tool = prediction.required_action.submit_tool_outputs.tool_calls[0]
#value = ast.literal_eval(tool.function.arguments)

xml = dict_to_xml_annotation(prediction)
ome_start_obj = ome_types.from_xml(ome_types.to_xml(OME()))
print(ome_start_obj.structured_annotations)
ome_start_obj.structured_annotations.append(xml)
print(ome_start_obj.structured_annotations)
print(ome_types.to_xml(ome_start_obj.structured_annotations))

queued
requires_action
RequiredActionFunctionToolCall(id='call_On0XF96qnNWrrgTKvCoYgrWN', function=Function(arguments='{"annotations":{"ImageProperties":{"Resolution":{"XResolution":"2728469.0","YResolution":"2728469.0","ResolutionUnit":"Inch"},"Dimensions":{"ImageWidth":"3072","ImageLength":"3072"},"Photometric":{"PhotometricInterpretation":"BlackIsZero","MetaDataPhotometricInterpretation":"Monochrome"},"Configuration":{"PlanarConfiguration":"Chunky","Orientation":"1st row -> top; 1st column -> left"},"Compression":"LZW","BitsPerSample":"16","SamplesPerPixel":"1","NumberOfChannels":"1","MetaMorph":"no"}}}', name='XMLAnnotationFunction'), type='function')
{'annotations': {'ImageProperties': {'Resolution': {'XResolution': '2728469.0', 'YResolution': '2728469.0', 'ResolutionUnit': 'Inch'}, 'Dimensions': {'ImageWidth': '3072', 'ImageLength': '3072'}, 'Photometric': {'PhotometricInterpretation': 'BlackIsZero', 'MetaDataPhotometricInterpretation': 'Monochrome'}, 'Configuration': {'PlanarCon

In [2]:
# network approach
from src.metagpt.predictors.predictor_annotation_net import PredictorXMLAnnotationNet
from ome_types._autogenerated.ome_2016_06 import XMLAnnotation

predictor = PredictorXMLAnnotationNet(str(ome_tree))
prediction = predictor.predict()
print(prediction)

queued
requires_action
{"prop": {"XResolution": "2728469.0", "PhotometricInterpretation": "BlackIsZero", "PlanarConfiguration": "Chunky", "MetaDataPhotometricInterpretation": "Monochrome", "ImageLength": "3072", "BitsPerSample": "16", "ResolutionUnit": "Inch", "Compression": "LZW", "SamplesPerPixel": "1", "NumberOfChannels": "1", "MetaMorph": "no", "ImageWidth": "3072", "Orientation": "1st row -> top; 1st column -> left", "YResolution": "2728469.0"}}
queued
requires_action
{'xml_annotations': '<ImageMetadata><Resolution><XResolution>2728469.0</XResolution><YResolution>2728469.0</YResolution><ResolutionUnit>Inch</ResolutionUnit></Resolution><Dimensions><ImageWidth>3072</ImageWidth><ImageLength>3072</ImageLength></Dimensions><Photometric><PhotometricInterpretation>BlackIsZero</PhotometricInterpretation><MetaDataPhotometricInterpretation>Monochrome</MetaDataPhotometricInterpretation></Photometric><Configuration><PlanarConfiguration>Chunky</PlanarConfiguration><Orientation>1st row -> top; 